# Laboratorio 11

## Integrantes:
- Alexander Sacchetti
- Andrés Calderón
- Felipe Fierro

Para empezar a utilizar mongodb desde 0, vamos a partir realizando los siguientes pasos:


1.   Instalaremos $\texttt{mongodb}$
2.   Inicializaremos el proceso $\texttt{mongodb}$
3.   Instalaremos el comando $\texttt{gdown}$ para bajar la base de datos en formato json de google drive.
4.   Importaremos los datos por linea de comando. Dejaremos los datos en la base de datos $\texttt{local}$, en la colección $\texttt{pokedex}$, y le informaremos al comando que los datos vienen como una arreglo de jsons.




In [1]:
!wget -qO- \
  https://pgp.mongodb.com/server-7.0.asc | \
  gpg --dearmor | \
  sudo tee /usr/share/keyrings/mongodb-server-7.0.gpg >/dev/null
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ] \
  https://repo.mongodb.org/apt/ubuntu $(lsb_release -cs)/mongodb-org/7.0 multiverse" | \
  sudo tee -a /etc/apt/sources.list.d/mongodb-org-7.0.list
!apt update
!apt install mongodb-org
!mkdir /data
!mkdir /data/db
!mongod --fork --logpath /var/log/mongodb/mongod.log
!gdown https://drive.google.com/uc?id=108aLE4mFE5bZYitFfX5c14Qj_aYA5MO_
!mongoimport --db local --collection pokedex --jsonArray < pokedex.json
!pip install pymongo

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ]    https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 multiverse
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Ign:2 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [47.2 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [632 kB]
Get:6 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 Release [2,090 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 Release.gpg [866 B]
Get:9 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:11 https://repo.mongodb.org/apt/u

In [2]:
!wget -qO- \
  https://pgp.mongodb.com/server-7.0.asc | \
  gpg --dearmor | \
  sudo tee /usr/share/keyrings/mongodb-server-7.0.gpg >/dev/null
!echo "deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ] \
  https://repo.mongodb.org/apt/ubuntu $(lsb_release -cs)/mongodb-org/7.0 multiverse" | \
  sudo tee -a /etc/apt/sources.list.d/mongodb-org-7.0.list

deb [ arch=amd64,arm64 signed-by=/usr/share/keyrings/mongodb-server-7.0.gpg ]    https://repo.mongodb.org/apt/ubuntu jammy/mongodb-org/7.0 multiverse


A continuación importaremos la librería de mongo para python, crearemos un cliente que se conectara a $\texttt{localhost}$ por defecto, nos conectaremos a la base local, y luego a la colección pokedex la cual almacenaremos en la variable $\texttt{collection}$.

In [3]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']
db = client['local']
collection = db['pokedex']

Para partir, podemos listar toda la colección usando el comando $\texttt{collection.find({})}$. Para no saturar la salida, nos limitaremos a mirar el primero de los registros.

In [ ]:
import pprint
for pokemon in collection.find({}).limit(1):
    pprint.pprint(pokemon)

Fijense que aparte de los datos básicos del pokemon, también se presentan las habilidades (una lista de strings), y los movimientos (un arreglo asociativo/diccionario de los movimientos y sus detalles).

Si queremos ahora listar los pokemones que se llaman $\texttt{Charmander}$ podemos hacer la siguiente consulta:

In [ ]:
for pokemon in collection.find({'Name': 'Charmander'}):
    pprint.pprint(pokemon)

Y si queremos proyectar algunos de sus atributos lo hacemos tal como si fuera un diccionario:

In [6]:
for pokemon in collection.find({'Name': 'Mew'}):
    print(pokemon['Speed'],pokemon['Abilities'])

100 ['Synchronize']


Recomendamos hechar un vistazo [aquí](https://docs.mongodb.com/manual/tutorial/query-documents/) para ver otros ejemplos de consultas que utilizarán en las siguientes preguntas:

#Pregunta 1
Encuentre los nombre de los pokemons que pesen mas de 150kg.
[Ordene](https://www.w3schools.com/python/python_mongodb_sort.asp) los resultados por peso descendiente

In [ ]:
for pokemon in collection.find({'Weight (kg)': {'$gt': 150}}).sort('Weight (kg)', -1):
  print(pokemon['Name'])

#Pregunta 2
Encuentre los nombres de los pokemones ''Glass cannon'', que son los que tienen ataque sobre 120 y defensa bajo 50.

In [17]:
for pokemon in collection.find({
    'Attack': {'$gt': 120},
    'Defense': {'$lt': 50}
}):
  print(pokemon['Name'])

Cranidos
Pheromosa


#Pregunta 3
Encuentre los pokemones "defensivos", que son los que tienen ya sea defensa, hp, o defensa especial (sp. defense) mayor o igual a 120.

In [ ]:
for pokemon in collection.find({
    '$or': [{'HP': {'$gte': 120}}, {'Defense': {'$gte': 120}}, {'Sp. Defense': {'$gte': 120}}]
}):
  pprint.pprint(pokemon)

#Pregunta 5
Seleccione los pokemones que tienen el movimiento $\texttt{Take Down}$ de nivel 15.

**Hint**: Puede consultar en diccionarios embebidos concatenando las llaves con un punto ".".

Por ejemplo, para un dato:
```
[{
  a: {
    b: 'foo'
  }
}]
```
se puede consultar por los documentos tal que $\texttt{b}$ sea $\texttt{foo}$, escribiendo $\texttt{collection.find({`a.b', `foo'})}$.

In [ ]:
for pokemon in collection.find({
    'Moves.Take Down.Level': 15
}):
  pprint.pprint(pokemon)

#Pregunta 4
Seleccione los pokemones que tienen el movimiento $\texttt{Take Down}$.

**Hint**: Puede testear si es que una llave existe utilizando [$exists](https://docs.mongodb.com/manual/reference/operator/query/exists/)

In [ ]:
for pokemon in collection.find({
    'Moves.Take Down': {'$exists': True}
}):
  pprint.pprint(pokemon)

#Pregunta 5
1. Seleccione los pokemones que tienen la habilidad $\texttt{Telepathy}$


In [36]:
for pokemon in collection.find({
    'Abilities': ['Telepathy']
}):
  pprint.pprint(pokemon)

2. Seleccione los pokemones que tienen la habilidad $\texttt{Telepathy}$
 **y** $\texttt{Synchronize}$ (pueden tener mas habilidades aparte de estas dos.
   
   **Hint**: puede utilizar el operador [$all](https://docs.mongodb.com/manual/tutorial/query-arrays/) .

In [32]:
for pokemon in collection.find({
    'Abilities': {'$all': ['Telepathy', 'Synchronize']}
}):
  print(pokemon['Name'])

Ralts
Kirlia
Gardevoir
Munna
Musharna
Elgyem
Beheeyem


#Pregunta 6
Para la siguiente pregunta vamos a crear un índice sobre la columna $\texttt{Category}$.

In [37]:
collection.create_index([('Category', 'text')])

'Category_text'

Usando el operador [$text](https://www.mongodb.com/docs/manual/reference/operator/query/text/#examples) responda a las siguientes consultas.
1.  Devuelva los nombres y categorías de los pokemons tales que su categoría contenga ya sea la palabra **Sea** o **Gas**.

In [ ]:
for pokemon in collection.find({
    '$text': {'$search': 'Sea Gas'}
}):
  print(pokemon['Name'], pokemon['Category'])

2. Devuelva los nombres y categorías de los pokemons tales que su categoría contenga la palabra **Sea** y no **Deep**.

In [ ]:
for pokemon in collection.find({
    '$text': {'$search': 'Sea -Deep'}
}):
  print(pokemon['Name'], pokemon['Category'])

3. Devuelva los nombres y categorías de los pokemons tales que su categoría contenga ambas palabras **Sea** y **Lion**.

In [49]:
for pokemon in collection.find({
    '$text': {'$search': '\"Sea\" \"Lion\"'}
}):
  print(pokemon['Name'], pokemon['Category'])

Popplio Sea Lion Pokémon
Dewgong Sea Lion Pokémon
Seel Sea Lion Pokémon
